In [3]:
import pandas as pd
import sqlite3
import os
import openpyxl
os.chdir(r"C:\Users\User\Documents\GitHub\python_homework\lesson-17\homework")

with sqlite3.connect("../data/chinook.db") as connection:
    df_customers = pd.read_sql(
        "SELECT * FROM customers",
        con=connection
    )
    df_invoices = pd.read_sql(
        "SELECT * FROM invoices",
        con=connection
    )



## inner join ##

In [4]:
inner_join = pd.merge(df_customers, df_invoices, on="CustomerId", how="inner")
total_invoices = inner_join.groupby(["CustomerId", "FirstName", "LastName"]).size().reset_index(name="TotalInvoices")
total_invoices.head()

,CustomerId,FirstName,LastName,TotalInvoices
0,1,Luís,Gonçalves,7
1,2,Leonie,Köhler,7
2,3,François,Tremblay,7
3,4,Bjørn,Hansen,7
4,5,František,Wichterlová,7


## outer join ##

In [5]:
df_movie = pd.read_csv("../data/movie.csv")
df_movie_1 = df_movie[["director_name", "color"]]
df_movie_2 = df_movie[["director_name", "num_critic_for_reviews"]]
left_join = pd.merge(df_movie_1, df_movie_2, on="director_name", how="left")
outer_join = pd.merge(df_movie_1, df_movie_2, on="director_name", how="outer")
print(f"Left join rows: {left_join.shape[0]}")
print(f"Outer join rows: {outer_join.shape[0]}")

Left join rows: 30300
Outer join rows: 30300


## grouped aggregations on titanic ##

In [6]:
df_titanic = pd.read_excel("../data/titanic.xlsx")
df_titanic_info = df_titanic.groupby("Pclass").agg({
    "Age":"mean",
    "Fare":"sum",
    "PassengerId":"count"
}).reset_index()
df_titanic_info.columns = ["Pclass", "Age", "Fare", "TotalPassengers"]
df_titanic_info

,Pclass,Age,Fare,TotalPassengers
0,1,38.233441,18177.4125,216
1,2,29.877630,3801.8417,184
2,3,25.140620,6714.6951,491


## multi-level grouping on movie data ##

In [7]:
df_multi_level_grouping = df_movie.groupby(["color", "director_name"]).agg(
    {
        "num_critic_for_reviews":"sum",
        "duration":"mean"
    }
).reset_index()
df_multi_level_grouping

,color,director_name,num_critic_for_reviews,duration
0,Black and White,Akira Kurosawa,153.0,202.0
1,Black and White,Aleksey German,121.0,177.0
2,Black and White,Alex Garland,489.0,108.0
3,Black and White,Alexander Payne,433.0,115.0
4,Black and White,Alfred Hitchcock,434.0,119.0
...,...,...,...,...
2485,Color,Zoran Lisinac,17.0,108.0
2486,Color,Álex de la Iglesia,71.0,104.0
2487,Color,Émile Gaudreault,67.0,92.0
2488,Color,Éric Tessier,9.0,99.0


## nested grouping on flights ##

In [ ]:
df_flights = pd.read_parquet("../data/flights")
df_nested_group_flights =df_flights.groupby(["Year", "Month"]).agg(
    {
        "FlightId":"count",
        "ArrDelay":"mean",
        "DepDelay":"max"

    }
)


## apply a custom function on titanic ##

In [26]:
def child_or_adult(age):
    if not (age>0): return
    elif age<18: return "Child"
    else: return "Adult"
df_titanic["Age_Group"] = df_titanic["Age"].transform(child_or_adult)
df_titanic.sample(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Group
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S,Adult
834,835,0,3,"Allum, Mr. Owen George",male,18.0,0,0,2223,8.3000,NaN,S,Adult
378,379,0,3,"Betros, Mr. Tannous",male,20.0,0,0,2648,4.0125,NaN,C,Adult
303,304,1,2,"Keane, Miss. Nora A",female,NaN,0,0,226593,12.3500,E101,Q,None


## normalize employee salaries ##

In [27]:
df_employee = pd.read_csv("../data/employee.csv")
def min_max_normalize(series):
    return (series-series.min())/(series.max()-series.min())
df_employee["NORMALIZED_SALARY"] = df_employee.groupby("DEPARTMENT")["BASE_SALARY"].transform(min_max_normalize)
df_employee.head(5)

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE,NORMALIZED_SALARY
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13,1.000000
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18,0.000000
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03,0.116351
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25,0.192491
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22,0.479189


## custom function on movies ##

In [68]:
def movie_duration(duration):
    if not duration>=0: return
    elif duration<60: return "Short"
    elif duration<120: return "Medium"
    else: return "Long"

df_movie["duration_type"] = df_movie["duration"].apply(movie_duration)
df_movie[["duration", "duration_type"]].sample(5)

,duration,duration_type
3495,88.0,Medium
276,22.0,Short
1894,158.0,Long
2443,114.0,Medium
935,96.0,Medium


## pipeline on titanic ##

In [87]:
def filter_passengers(df):
    return df[df["Survived"]==1].copy()
def fill_ages(df):
    df.loc[:,"Age"] = df["Age"].fillna(df["Age"].mean())
    return df
def new_column(df):
    df.loc[:,"Fare_Per_Age"] = df["Fare"] / df["Age"]
    return df

df_titanic_pipe = (df_titanic.pipe(filter_passengers).pipe(fill_ages).pipe(new_column))
df_titanic_pipe.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Group,Fare_Per_Age
237,238,1,2,"Collyer, Miss. Marjorie ""Lottie""",female,8.0,0,2,C.A. 31921,26.2500,NaN,S,Child,3.281250
838,839,1,3,"Chip, Mr. Chang",male,32.0,0,0,1601,56.4958,NaN,S,Adult,1.765494
580,581,1,2,"Christy, Miss. Julie Rachel",female,25.0,1,1,237789,30.0000,NaN,S,Adult,1.200000
550,551,1,1,"Thayer, Mr. John Borland Jr",male,17.0,0,2,17421,110.8833,C70,C,Child,6.522547
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0000,C23 C25 C27,S,Adult,11.434783


## pipeline on flights ##

In [ ]:
def filter_departure_delay(df):
    return df[df["Delay"]>30]
def delay_per_hour(df):
    df["Delay_Per_Hour"] = df["Delay"]/df["Duration"]
    return df
df_flights.pipe(filter_departure_delay).pipe(delay_per_hour)